<h1>FILTRAGEM NO DOMÍNIO DA FREQUÊNCIA</h1>

<h2>Sumário</h2>
    
[1. Introdução](#introducao)<br>
[2. Script de implementação do Filtro Homomórfico](#script1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.1 Bibliotecas](#bibliotecas)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2 Função de deslocamento da DFT](#funcao1)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3 Filtro Homórfico](#filtro)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.4 Trackbar para variação dos parâmetros](#trackbar)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.5 Inicialização das variáveis](#init)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.6 Loop de variação dos parâmetros](#loop)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.7 Resultados](#result)<br>

<a id='introducao'></a>

<h2>1. Introdução</h2>

<p style='text-align: justify;'>Este exercício tem com objetivo o entendimento acerca do processo de filtragem no domínio da frequência, onde pôde-se elucidar os conhecimentos relacionados à representação de imagens no domínio da frequência através da Transformada Discreta de Fourier (DFT). Para a execução do processo de filtragem, foi implementado um filtro homomórfico para realizar a correção de problemas de iluminação em imagens.</p>

<a id='script1'></a>

<h2>2. Script de implementação do Filtro Homomórfico</h2>

<p style='text-align: justify;'>Esta seção descreverá a implementação de um algoritmo capaz de realizar o processo de filtragem através de um Filtro Homomórfico com parâmetros variáveis.</p>

<a id='bibliotecas'></a>

<h3>2.1 Bibliotecas</h3>

<p style='text-align: justify;'>Inicialmente, importa-se as bibliotecas necessárias para a realização dos algoritmos. A primeira consiste na biblioteca do openCV para Python, utilizada para todo o tratamento das imagens e realização de processos como por exemplo, a DFT. O numpy é utilizado para trabalhar com as matrizes das imagens, possibilitando operações matemáticas, por exmeplo.</p>

In [12]:
import cv2
import numpy as np

<a id='funcao1'></a>

<h3>2.2 Função de deslocamento da DFT</h3>

A função abaixo recebe uma matriz multidimensional como parâmetro e realiza a inversão dos quadrantes para garantir uma melhor representação da transformada. A matriz que entra na função tem os quadrantes da seguinte forma:

<table>
<tr>
    <td>A</td>
    <td>B</td>
</tr>
<tr>
    <td>C</td>
    <td>D</td>
</tr>
</table>

Já na saída, os quadrantes serão reorganizados da seguinte forma:

<table>
<tr>
    <td>D</td>
    <td>C</td>
</tr>
<tr>
    <td>B</td>
    <td>A</td>
</tr>
</table>

In [13]:
def deslocaDFT(img):
    A = img[0:np.shape(img)[0]//2,0:np.shape(img)[1]//2,:]
    B = img[0:np.shape(img)[0]//2,np.shape(img)[1]//2:np.shape(img)[1],:]
    C = img[np.shape(img)[0]//2:np.shape(img)[0],0:np.shape(img)[1]//2,:]
    D = img[np.shape(img)[0]//2:np.shape(img)[0],np.shape(img)[1]//2:np.shape(img)[1],:]

    img1 = np.copy(img)
    img1[0:np.shape(img)[0]//2,0:np.shape(img)[1]//2,:] = D
    img1[0:np.shape(img)[0]//2,np.shape(img)[1]//2:np.shape(img)[1],:] = C
    img1[np.shape(img)[0]//2:np.shape(img)[0],0:np.shape(img)[1]//2,:] = B
    img1[np.shape(img)[0]//2:np.shape(img)[0],np.shape(img)[1]//2:np.shape(img)[1],:] = A
    
    return img1

<a id='filtro'></a>

<h3>2.3 Filtro Homomórfico</h3>

Para corrigir a má iluminação de uma cena, um filtro homomórfico pode ser utilizado. Sua implementação no domínio da frequência é possível através da seguinte equação:

$$H(u,v)=(\gamma_H-\gamma_L)\left(1-e^{-c\left(\frac{D^2(u,v)}{D^2_0}\right)}\right)+\gamma_L$$

Dessa forma, tem-se os parâmetros $\gamma_L$, $\gamma_H$, $c$ e $D_0$ como parâmetros para a regulação do filtro. Logo, a equação acima foi transformada em uma função, a qual recebe como parâmetros as variáveis indicadas anteriormente e as dimensões da DFT da imagem. Dessa forma, um filtro com parte real e imaginária é gerado e retornado.

In [14]:
def filtro_homomorfico(gammaH, gammaL, D0, c, dft_M, dft_N):
    filtro = np.zeros([dft_M,dft_N,2], dtype='float32')
    for i in range(dft_M):
        for j in range(dft_N):
            filtro[i][j][:] = (gammaH-gammaL)*(1-np.exp(-c*(((i-dft_M/2)**2 + (j-dft_N/2)**2) / (D0**2)))) + gammaL
    return filtro

<a id='trackbar'></a>

<h3>2.4 Trackbar para variação dos parâmetros</h3>

Para que os parâmetros pudessem ser todos variados em tempo de execução do programa, foram criados quatro <i>Trackbars</i>, sendo uma para cada parâmetro variável do filtro. Algumas condições foram consideradas para o bom funcionamento do código, como por exemplo, impedir que a variável $D_0$, que está presente em um denominador, seja zero, ocasionando em um resultado infinito. O trecho de código das funções dos <i>Trackbars</i> são mostradas abaixo:

In [15]:
def on_trackbar_gammaH(gammaH_slider):
    global gammaH
    global gammaL
    if gammaH > gammaL:
        gammaH = gammaH_slider
    

def on_trackbar_gammaL(gammaL_slider):
    global gammaL
    global gammaH
    if gammaL < gammaH:
        gammaL = gammaL_slider
    
def on_trackbar_D0(D0_slider):
    global D0
    if D0_slider != 0:
        D0 = D0_slider
    
def on_trackbar_c(c_slider):
    global c
    c = c_slider

<a id='init'></a>

<h3>2.5 Inicialização das variáveis</h3>

Neste trecho de código, são inicializadas as variáveis úteis ao programa. Nesta parte, lê-se a imagem alvo do processamento, bem como calcula-se a frequência máxima desta imagem. Além disso, os limites das <i>Trackbars</i> são definidos e estas são criadas, assim como é gerada a imagem com as bordas completadas com zeros (<i>padding</i>) para otimização da DFT.

In [1]:
image = cv2.imread("..\\images\\surprise.jpeg", 0)

dft_M = cv2.getOptimalDFTSize(image.shape[0])
dft_N = cv2.getOptimalDFTSize(image.shape[1])

freq_max = int(dft_M/2 - 1)

cv2.namedWindow("Filtrada",1)

gammaH = 15
gammaH_max = freq_max
gammaH_slider = 15

gammaL = 10
gammaL_max = freq_max-1
gammaL_slider = 10

D0 = 10
D0_max = freq_max
D0_slider = 10

c = 10
c_max = freq_max*10
c_slider = 10

cv2.createTrackbar("gammaH","Filtrada", gammaH_slider, gammaH_max, on_trackbar_gammaH)
on_trackbar_gammaH(gammaH)

cv2.createTrackbar("gammaL","Filtrada", gammaL_slider, gammaL_max, on_trackbar_gammaL)
on_trackbar_gammaL(gammaL)

cv2.createTrackbar("D0","Filtrada", D0_slider, D0_max, on_trackbar_D0)
on_trackbar_D0(D0)

cv2.createTrackbar("c","Filtrada", c_slider, c_max, on_trackbar_c)
on_trackbar_c(c)

padded = cv2.copyMakeBorder(image, 0, dft_M - image.shape[0], 0, dft_N - image.shape[1], cv2.BORDER_CONSTANT, 0)

<a id='loop'></a>

<h3>2.6 Loop de variação dos parâmetros</h3>

Ao executar o código, será exibida a imagem resultante do processo de filtragem e as barras de configuração dos parâmetros do filtro. A cada mudança nas barras, a imagem é recriada já com a nova filtragem. No loop, a DFT da imagem é calculada e multiplicada pela resposta do filtro. Em seguida, a transformada inversa é utilizada para a recuperação da imagem filtrada.

In [11]:
while(1):   
    filtro = filtro_homomorfico(gammaH, gammaL, D0, c, dft_M, dft_N)
    
    dft = cv2.dft(np.float32(padded), flags = cv2.DFT_COMPLEX_OUTPUT)
    dft = deslocaDFT(dft)
    
    dft = np.multiply(dft,filtro)
    
    dft = deslocaDFT(dft)
    
    real = cv2.idft(dft, flags= cv2.DFT_REAL_OUTPUT)
    real = cv2.normalize(real, None, 0, 1, cv2.NORM_MINMAX)
    cv2.imshow("Filtrada", real)
    
    #mag, ang = cv2.cartToPolar(filtro[:,:,0],filtro[:,:,1])
    #mag = mag + 1
    #mag = np.log(mag)
    #mag = cv2.normalize(mag, None, 0, 1, cv2.NORM_MINMAX)
    #cv2.imshow("Filtro", mag)    

    k = chr(cv2.waitKey(10) & 0xff)
    if ord(k) == 27:
        cv2.destroyAllWindows()
        break

<a id='result'></a>

<h3>2.7 Resultados</h3>

Para a cena a ser processa, utilizou-se um flash de câmera de celular para causar uma iluminação muito forte, como mostrado abaixo:

![sup_gray](../images/documentation/surprise_gray.jpg)

Após um certo tempo variando os parâmetros, obteve-se o seguinte resultado para $\gamma_L=213$, $\gamma_H=225$, $c=520$ e $D_0=61$:

![sup](../images/documentation/surprise.jpg)

A expectativa era tornar um brinquedo do pikachu mais aparente na região mais escura à esquerda, porém, ficou meio assustador. A figura abaixo destaca a posição do brinquedo:

![pikachu](../images/documentation/pikachu.jpg)

A resposta do filtro com os parâmetros mostrados acimas foi a seguinte:

![resp_homo](../images/documentation/resp_homo.jpg)
